# creation of command history
this notebook will create an historic of command, each lines represent an user which commanded a recipe

__TABLE Command__:

| user_id           | recepie_id      | 
|-------------------|-----------------|
|PRIMARY KEY        |PRIMARY KEY      | 

Those transaction won't be created randomly, I will use the favourite recepie of the user, TF-IDF on the description of this recepie and all the others, and a random factor to select which recepie the user commanded.

## build the distance matrix
we will use the recepie instructions and summary to be able to determine which are closer to the others with tf-idf

In [94]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

get the data from the db:

In [95]:
conn = sqlite3.connect('../Data/Supermarket.db')

query = "SELECT id, instructions, summary FROM Recipes"

df = pd.read_sql_query(query, conn)

conn.commit()
conn.close()
df

,id,instructions,summary
0,715495,Heat up your grill to 450 degrees.Start off wi...,Turkey Tomato Cheese Pizza might be just the <...
1,665282,"In a small saucepan, bring the 1 cup water and...","You can never have too many bread recipes, so ..."
2,632197,<ol><li>1. Preheat oven to 350 degrees (325 if...,Almond Toffee Bars is a dessert that serves 1....
3,658536,Preheat oven to 425 degrees Farenheit.\nSpread...,Roasted Cauliflower and Leek Soup is a <b>glut...
4,639836,Pat and dry fish fillets. Sprinkle both sides ...,Coconut-Almond Crusted Tilapia requires approx...
...,...,...,...
1005,716202,Soak the dried fenugreek leaves in warm water ...,Fenugreek Roti requires roughly <b>45 minutes<...
1006,632071,<ol><li>Heat the olive oil in a large skillet....,The recipe All American Sloppy Joes could sati...
1007,648474,"In a pan over medium heat, add oil, garlic and...",Japanese Fried Rice is a <b>gluten free and da...
1008,662376,<ol><li>In a large heavy pot set over high hea...,If you want to add more <b>gluten free</b> rec...


create a column with summary + instruction (in text format not html)

In [96]:
df['instructions'] = df['instructions'].astype('str')
df['summary'] = df['summary'].astype('str')

In [97]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text()
    return cleaned_text


In [98]:
df['description'] = df['summary'].apply(remove_html_tags) + df['instructions'].apply(remove_html_tags)

C:\Users\sacha\AppData\Local\Temp\ipykernel_17168\3658179466.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


preprocess the column

In [99]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Charger les stopwords
nltk.download('stopwords')
nltk.download('punkt')

# Définir les stopwords et la ponctuation à supprimer
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sacha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sacha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [100]:
def pre_process_text(text: str)-> str:
    """ 
    the function will preprocess and return the overview
    Lowercase, delete stop words and punctuation
    """
    global stop_words
    global punctuation

    # Tokenisation des mots
    tokens = word_tokenize(text.lower())
    # Suppression des stopwords et de la ponctuation
    tokens = [w for w in tokens if not w in stop_words and not w in punctuation]
    
    # Rejoindre les mots restants en un seul texte
    cleaned_text = " ".join(tokens)
    return cleaned_text

In [101]:
df['description']= df['description'].apply(pre_process_text)

tfidf matrix:

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer

description_list = df['description'].tolist()

tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(description_list)

print("matrix size :", tfidf_matrix.shape)

matrix size : (1010, 6818)


cosine similarity:

In [103]:
from sklearn.metrics.pairwise import cosine_similarity
#30 sec

tfidf_matrix = tfidf_matrix.astype('float32')  #takes less time

similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("matrix size :", similarity_matrix.shape)

matrix size : (1010, 1010)


## create the differents command
for each users: if random(0,1) < similarity(fav_recepie) then add a command

In [104]:
conn = sqlite3.connect('../Data/Supermarket.db')

query = "SELECT * FROM Users"

df_users = pd.read_sql_query(query, conn)

conn.commit()
conn.close()

In [105]:
import random

In [106]:
list_rec = []
list_user = []

for i in range(len(df_users)):
    #get the favourite recepie of the client
    fav_rec_id = df_users.iloc[i]['recepie_id']
    #get the client's id
    user_id = df_users.iloc[i]['id']
    
    #get the position of the recepie in the matrix
    position = df[df['id'] == fav_rec_id].index[0]

    #add the recepie that respect the treshold
    treshold = random.random()
    for r in range(len(df)):
        similarity_coef = similarity_matrix[position][r]

        if treshold < similarity_coef:
            recepie_id = df.iloc[r]['id']
            list_rec.extend([recepie_id])
            list_user.extend([user_id])

df_command = pd.DataFrame({'user_id': list_user, 'recepie_id': list_rec })

## add it to the database

In [107]:
#push this dataset as a table in the db
conn = sqlite3.connect('../Data/Supermarket.db')

df_command.to_sql('Commands', conn, if_exists='replace', index=False)

conn.commit()
conn.close()